In [ ]:
from datetme import date, timedelta

In [ ]:
start_date = date.today() - timedelta(1)

silver_adls = 'abfss://silver@eqstorage.dfs.core.windows.net/'
gold_adls = 'abfss://gold@eqstorage.dfs.core.windows.net/'

silver_data = f'{silver_adls}earthquake_events_silver/'

In [ ]:
from pyspark.sql import functions import when, col, udf
from pyspark.sql.types import StringType
import reverse_geocoder as rg
from datetime import date, timedelta

In [ ]:
df = spark.read.parquet(silver_data).filter(col('time') > start_date)


In [ ]:
df = df.limit(100)


In [ ]:
def get_country_code(lat, lon):
    """
    Retrieve the country code for a given latitude and longitude.

    Parameters:
    lat (float or str): Latitude of the location.
    lon (float or str): Longitude of the location.

    Returns:
    str: Country code of the location, retrieved using the reverse geocoding API.

    Example:
    >>> get_country_details(48.8588443, 2.2943506)
    'FR'
    """
    try:
        coordinates = (float(lat), float(lon))
        result = rg.search(coordinates)[0].get('cc')
        return result
    except Exception as e:
        print(f'Error processing coordinates: {lat}, {lon} -> {str(e)}')
        return None


In [ ]:
#udf stands for user defined function its a way to use custom functions in spark dataframes
get_country_code_udf = udf(get_country_code, StringType()) 

get_country_code(48.8588443, 2.2943506)

In [ ]:
#adding country code and city attributes
df_with_location = \
        df.\
            withColumn('country_code', get_country_code_udf(col('latitude'), col('longitude')))

In [ ]:
df_with_location.printSchema()


In [ ]:
#adding significance category
df_with_location_sig_class = \
    df_with_location.\
        withColumn('sig_class',
            withColumn('sig_class',
                when(col('sig') < 100, 'Low').\
                when((col('sig') >= 100) & (col('sig') < 500), 'Moderate').\
                otherwise('High')
            )

df_with_location_sig_class.printSchema()

In [ ]:
#Save the transformed dataframe to the gold container
gold_output_path = f'{gold_adls}earthquake_events_gold/'


In [ ]:
df_with_location_sig_class.write.mode('append').parquet(gold_output_path)